## Apply julia-decisiontree to Titanic Dataset

- Load dependencies
- Load training and test data
- Transform data: ignore features cabin, age, and embarked. Normalise gender? Group features?
- Create Classifier
- Train model
- Assess performance: accuracy, overfitting

In [170]:
import Pkg;
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("DecisionTree")

using DataFrames
using CSV
using DecisionTree

 Resolving package versions...
  Updating `/opt/julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/opt/julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/opt/julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.1/Manifest.toml`
 [no changes]


In [171]:
train = CSV.read("data/train.csv")
test = CSV.read("data/test.csv")
describe(train)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Int64,DataType
1,PassengerId,446.0,1,446.0,891,,0,Int64
2,Survived,0.383838,0,0.0,1,,0,Int64
3,Pclass,2.30864,1,3.0,3,,0,Int64
4,Name,,"Abbing, Mr. Anthony",,"van Melkebeke, Mr. Philemon",891,0,String
5,Sex,,female,,male,2,0,String
6,Age,29.6991,0.42,28.0,80.0,,177,Float64
7,SibSp,0.523008,0,0.0,8,,0,Int64
8,Parch,0.381594,0,0.0,6,,0,Int64
9,Ticket,,110152,,WE/P 5735,681,0,String


In [172]:
first(train, 5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64⍰,Int64⍰,Int64⍰,String⍰,String⍰,Float64⍰,Int64⍰,Int64⍰,String⍰,Float64⍰,String⍰,String⍰
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,missing,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,missing,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,missing,S


In [173]:
df = train[:,[:Pclass, :Sex, :SibSp, :Parch, :Fare, :Survived]]


first(df, 5)

,Pclass,Sex,SibSp,Parch,Fare,Survived
,Int64⍰,String⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰
1,3,male,1,0,7.25,0
2,1,female,1,0,71.2833,1
3,3,female,0,0,7.925,1
4,1,female,1,0,53.1,1
5,3,male,0,0,8.05,0


In [174]:
function normaliseGender(x)
    if (x == "male")
        1
    else
        0
    end
end

#df[:normalised_gender] = map(normaliseGender, df[:Sex])

deletecols!(df, :Sex)

first(df, 5)

,Pclass,SibSp,Parch,Fare,Survived
,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰
1,3,1,0,7.25,0
2,1,1,0,71.2833,1
3,3,0,0,7.925,1
4,1,1,0,53.1,1
5,3,0,0,8.05,0


In [175]:
colwise(eltype, df)

5-element Array{Union,1}:
 Union{Missing, Int64}  
 Union{Missing, Int64}  
 Union{Missing, Int64}  
 Union{Missing, Float64}
 Union{Missing, Int64}  

In [176]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Int64,DataType
1,Pclass,2.30864,1,3.0,3,,0,Int64
2,SibSp,0.523008,0,0.0,8,,0,Int64
3,Parch,0.381594,0,0.0,6,,0,Int64
4,Fare,32.2042,0.0,14.4542,512.329,,0,Float64
5,Survived,0.383838,0,0.0,1,,0,Int64


In [177]:
dropmissing!(df)
#describe(df)

df[:Pclass] = convert(Array{Int64,1},df[:Pclass])
df[:SibSp] = convert(Array{Int64,1},df[:SibSp])
df[:Parch] = convert(Array{Int64,1},df[:Parch])
df[:Survived] = convert(Array{Int64,1},df[:Survived])

df[:Floored_fare] = map(floor, df[:Fare])
df[:Floored_fare] = convert(Array{Int64,1},df[:Floored_fare])
deletecols!(df, :Fare)


colwise(eltype, df)
#describe(df)

5-element Array{DataType,1}:
 Int64
 Int64
 Int64
 Int64
 Int64

In [178]:
features = convert(Matrix, df[[:Pclass, :SibSp, :Parch, :Floored_fare]])
label = convert(Array, df[:Survived])

891-element Array{Int64,1}:
 0
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 1
 0
 ⋮
 1
 1
 0
 0
 0
 0
 0
 0
 1
 0
 1
 0

In [179]:


n_subfeatures=0; max_depth=-1; min_samples_leaf=1; min_samples_split=2
min_purity_increase=0.0; pruning_purity = 1.0

model    =   build_tree(label, features,
                        n_subfeatures,
                        max_depth,
                        min_samples_leaf,
                        min_samples_split,
                        min_purity_increase)

n_folds = 3
accuracy = nfoldCV_tree(label, features,
                        n_folds,
                        pruning_purity,
                        max_depth,
                        min_samples_leaf,
                        min_samples_split,
                        min_purity_increase)

2×2 Array{Int64,2}:
 172  19
  37  69


Fold 1
Classes:  [0, 1]
Matrix:   
Accuracy: 0.8114478114478114
Kappa:    0.5731225296442687


2×2 Array{Int64,2}:
 166  11
  50  70

2×2 Array{Int64,2}:
 167  14
  47  69


Fold 2
Classes:  [0, 1]
Matrix:   
Accuracy: 0.7946127946127947
Kappa:    0.5499664654594233

Fold 3
Classes:  [0, 1]
Matrix:   
Accuracy: 0.7946127946127947
Kappa:    0.5453359098551962

Mean Accuracy: 0.8002244668911335


3-element Array{Float64,1}:
 0.8114478114478114
 0.7946127946127947
 0.7946127946127947

In [181]:
n_folds=3; n_subfeatures=2

n_subfeatures=-1; n_trees=10; partial_sampling=0.7; max_depth=-1
min_samples_leaf=5; min_samples_split=2; min_purity_increase=0.0

model    =   build_forest(label, features,
                          n_subfeatures,
                          n_trees,
                          partial_sampling,
                          max_depth,
                          min_samples_leaf,
                          min_samples_split,
                          min_purity_increase)

accuracy = nfoldCV_forest(label, features,
                          n_folds,
                          n_subfeatures,
                          n_trees,
                          partial_sampling,
                          max_depth,
                          min_samples_leaf,
                          min_samples_split,
                          min_purity_increase)

2×2 Array{Int64,2}:
 163  16
  50  68

2×2 Array{Int64,2}:
 165  16
  61  55

2×2 Array{Int64,2}:
 162  27
  48  60


Fold 1
Classes:  [0, 1]
Matrix:   
Accuracy: 0.7777777777777778
Kappa:    0.5120238984316654

Fold 2
Classes:  [0, 1]
Matrix:   
Accuracy: 0.7407407407407407
Kappa:    0.4146210356566922

Fold 3
Classes:  [0, 1]
Matrix:   
Accuracy: 0.7474747474747475
Kappa:    0.4306418219461699

Mean Accuracy: 0.755331088664422


3-element Array{Float64,1}:
 0.7777777777777778
 0.7407407407407407
 0.7474747474747475